In [ ]:
!pip install transformers datasets


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
!pip install -q transformers datasets accelerate bitsandbytes peft


loading hugging face token

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


importing tinyllama for training

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


uploading dataset


In [ ]:
# Example for Colab/Jupyter:
from google.colab import files
uploaded = files.upload()

Saving pidgin_firstMath_dataset.json to pidgin_firstMath_dataset.json


reading dataset and giving it an object name f

setting line for prompt and response

In [ ]:
import json
from datasets import Dataset

with open("pidgin_firstMath_dataset.json", "r", encoding="utf-8") as f:
    raw = json.load(f)
    print(raw[5])

# Combine prompt and response
lines = []

for item in raw:
    if "pidgin_prompt" in item and "pidgin_response" in item:
        lines.append({
            "text": f"User: {item['pidgin_prompt']}\nAI: {item['pidgin_response']}"
        })
    else:
        print(" Skipping invalid entry:", item)


dataset = Dataset.from_list(lines)


{'pidgin_prompt': "You and your senior brother get some money. If you add your money (x) and your brother's money (y) together, una get 7k. But if your money na two times your brother's money minus your brother's money, una get 8k. How much money una get individually?", 'pidgin_response': "How we go solve am:\n1. First, make una add una two equations together, like say una dey combine una strength to lift something heavy. When you add (x + y) and (2x - y), the 'y' go just vanish, like a debtor running away from his creditor. So, you go get 3x = 15.\n2. Now, divide both sides by 3. So, **x = 5k.**\n3. Carry that 5k go back to the first equation (x + y = 7). Since x na 5, then 5 + y = 7. So, if you shift the 5 go other side, **y = 2k.**\nFinal answer: You get 5k, and your brother get 2k. Una don settle accounts!"}
 Skipping invalid entry: {'pidgin_prompt': 'Solve simultaneously: 5x + 9y = 5, 6x + 4y = 7', 'pidpidgin_response': "Δ = -34. x = (5×4 - 9×7)/Δ = 1.265, y = (5×7 - 5×6)/Δ = -0.1

saving f into a text file mathwahala_corpus.text

In [ ]:
with open("mathwahala_corpus.txt", "w" , encoding="utf-8") as f:
  for line in train_data:
    f.write(line + "\n\n")

NameError: name 'train_data' is not defined

tokenizing with transformers dataset format

In [ ]:
from transformers import AutoTokenizer

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize_function(example):
    result = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    # Add 'labels' (same as input_ids for Causal LM training)
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])



Map:   0%|          | 0/1277 [00:00<?, ? examples/s]

fine tuning tinyllama with lora

In [ ]:
!pip install peft transformers datasets accelerate bitsandbytes trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType


set up loRA

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
import torch

bnb_config = dict(load_in_8bit=True, device_map="auto")

base_model = AutoModelForCausalLM.from_pretrained(model_id, **bnb_config)
base_model = prepare_model_for_kbit_training(base_model)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(base_model, peft_config)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="mathwahala_lora_v2",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_steps=50,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    fp16=True,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,12.073700
20,11.609700
30,7.338900
40,2.658300
50,0.936000
60,0.759300
70,0.745000
80,0.619900
90,0.618000
100,0.554600


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reen

TrainOutput(global_step=480, training_loss=0.9390947149445613, metrics={'train_runtime': 1845.4836, 'train_samples_per_second': 2.076, 'train_steps_per_second': 0.26, 'total_flos': 1.2188258861580288e+16, 'train_loss': 0.9390947149445613, 'epoch': 3.0})

In [ ]:
trainer.save_model("mathwahala_lora_v2")


In [ ]:
!zip -r mathwahala_lora_v2.zip mathwahala_lora_v2
from google.colab import files
files.download("mathwahala_lora_v2.zip")


  adding: mathwahala_lora_v2/ (stored 0%)
  adding: mathwahala_lora_v2/adapter_model.safetensors (deflated 8%)
  adding: mathwahala_lora_v2/training_args.bin (deflated 52%)
  adding: mathwahala_lora_v2/checkpoint-480/ (stored 0%)
  adding: mathwahala_lora_v2/checkpoint-480/optimizer.pt (deflated 8%)
  adding: mathwahala_lora_v2/checkpoint-480/adapter_model.safetensors (deflated 8%)
  adding: mathwahala_lora_v2/checkpoint-480/rng_state.pth (deflated 25%)
  adding: mathwahala_lora_v2/checkpoint-480/scaler.pt (deflated 60%)
  adding: mathwahala_lora_v2/checkpoint-480/scheduler.pt (deflated 55%)
  adding: mathwahala_lora_v2/checkpoint-480/training_args.bin (deflated 52%)
  adding: mathwahala_lora_v2/checkpoint-480/trainer_state.json (deflated 79%)
  adding: mathwahala_lora_v2/checkpoint-480/adapter_config.json (deflated 54%)
  adding: mathwahala_lora_v2/checkpoint-480/README.md (deflated 66%)
  adding: mathwahala_lora_v2/adapter_config.json (deflated 54%)
  adding: mathwahala_lora_v2/check

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig

base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load base tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", load_in_8bit=True)

# Load your fine-tuned LoRA weights
lora_model = PeftModel.from_pretrained(model, "mathwahala_lora_v2")
lora_model = lora_model.merge_and_unload()  # merge LoRA into base for faster inference
lora_model.eval()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:85: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear8bitLt(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear8bitLt(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear8bitLt(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear8bitLt(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSN

In [ ]:
import torch

def ask_mathwahala(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(lora_model.device)
    output = lora_model.generate(**inputs, max_new_tokens=200, do_sample=True, top_k=50, top_p=0.95)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
prompt = "Dem give us two points on a line: (-1,7) and (-7,-9). Find out how steep dis line be (wey we dey call 'slope') abeg explain am for 10 year old pikin"
response = ask_mathwahala(prompt)
print(response)


Dem give us two points on a line: (-1,7) and (-7,-9). Find out how steep dis line be (wey we dey call 'slope') abeg explain am for 10 year old pikin.
AI: To find de steepness for de line, we go to calculate de curvature (also called curvature radius). De curvature ra wey de line go bend down or bend up, without straighten out. For steepness, de curvature radius be the 'y' term of de line. So, go take de 'y' term, which we call 'y=-ma/λ', and divide it by de 'λ' term, which we call 'm'. De 'y' term go look like this: ma × 7 × 9 × 9 × 9 × 9 × 9 × 7 × 7 × 7 × 7 × 9 = 13744. So, steepness go be 13744 divided by 23 = 6.12. So, de steepness for de line wey go bend down even de b


In [ ]:
!pip install gradio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
def ask_mathwahala(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        top_p=0.95,
        top_k=50
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=ask_mathwahala,
    inputs=gr.Textbox(lines=4, placeholder="Type your Pidgin maths question here..."),
    outputs="text",
    title=" MathWahala.ai (Pidgin AI Tutor)",
    description="Ask any question in Nigerian Pidgin and get friendly, math-savvy answers!"
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5890e36157ffa394f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from google.colab import files
files.download("/content/mathwahala_lora_v2")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>